In [60]:
# created with love in cooperation with ASPIS
import numpy as np
import pandas as pd
import keras
import os
import matplotlib.pyplot as plt
import pandas as pd
from keras.models import Model
from sklearn.metrics import confusion_matrix, classification_report, mean_absolute_error, roc_curve, roc_auc_score

from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, precision_recall_curve, plot_precision_recall_curve
from sklearn.metrics import fbeta_score, auc, confusion_matrix, classification_report, mean_absolute_error, roc_curve, roc_auc_score

In [61]:
df=pd.DataFrame(columns=['col', 'epoch', 'eval_v', 'tp', 'fp', 'fn', 'tn', 'p', 'r', 'f1', 'auc-roc', 'auc-pr', 'tss'])

In [62]:
def plot_roc_curve(fpr, tpr):
    plt.plot(fpr, tpr, color='orange', label='ROC')
    plt.plot([0, 1], [0, 1], color='darkblue', linestyle='--')
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('Receiver Operating Characteristic (ROC) Curve')
    plt.legend()
    plt.show()

In [63]:
print("###################")
print("# settings")
directory = "full-not-divided"
# nastav spravne cesty
for subdir, dirs, files in os.walk(f"/home/jovyan/data/lightning/adrian/dip/vierka_train/training/multiNN/"):
    for file in files:
        if "epoch-05" in file:
########################################################################################## 
#             settings
            col = subdir.split('/')[-1]
            print(col)
            _features_cols1 = ['f10.7_index','Kp_index', 'Dst_index', 'ap_index']
            _features_cols2 = ['E','BzGSE','Xbsn','AE','PC','AsyD']
            _features_cols3 = ['AsyH','SymH','O2[cm^-3]','N2[cm^-3]']
            _features_cols4 = ['Ti','ne','NmE', 'S4_Sig1','p_Sig1']
            
            if col in _features_cols1:
                f_name = "full-multinn"
            elif col in _features_cols2:
                f_name = "full-multinn-E-AsyD"
            elif col in _features_cols3:
                f_name = "full-multinn-AsyH-N2"
            elif col in _features_cols4:
                f_name = "full-multinn-Ti-p_Sig1"
            else:
                continue
            saved_model = "../training/multiNN/{}/{}".format(col, file)
########################################################################################## 
#             load data
            X_test = np.load(f'../data-preparation/{f_name}/shift-15-windows-45/X_test.npy',allow_pickle=True).item()
            y_test = np.load(f'../data-preparation/{f_name}/shift-15-windows-45/y_test.npy')
        
            Phi60_Sig1 = X_test["Phi60_Sig1"]
            Kp_index = X_test[col]

########################################################################################## 
            # predict
            model = keras.models.load_model(saved_model)

            y_pred = model.predict([Phi60_Sig1,Kp_index])
    
            y_true =  np.where(y_test >= 0.1, 1, 0)
            y_scores = y_pred

            fpr, tpr, thresholds = roc_curve(y_true, y_scores)

            optimal_idx = np.argmax(tpr - fpr)
            optimal_threshold = thresholds[optimal_idx]


########################################################################################## 
            print("# eval v1")
            y_pred2 = np.where(y_pred >= optimal_threshold, 1, 0)
            y_test2 = np.where(y_test >= 0.1, 1, 0)

            cm = confusion_matrix(y_test2, y_pred2)
            tn = cm[0][0]
            fp = cm[0][1]
            fn = cm[1][0]
            tp = cm[1][1]
            
            precision = float(tp/(tp+fp))
            recall = float(tp/(tp+fn))
            f1 = float((2*precision*recall)/(precision+recall))
            auc_roc=str(round(roc_auc_score(y_test2, y_pred), 4))
            precision_, recall_, _ = precision_recall_curve(y_test2, y_pred)
            auc_pr=str(round(auc(recall_, precision_), 4))
            tss = float((tp/(tp+fn)) - (fp/(fp+tn)))
            
            
            df.loc[len(df)] = [col, file, '1', tp, fp, fn, tn, round(precision, 4), round(recall, 4), round(f1, 4), auc_roc, auc_pr, round(tss, 4)]

            
########################################################################################## 
            print("# evaluation v2")

            df_pred = pd.DataFrame(y_pred)
            df_test = pd.DataFrame(y_test)

            df_test["y_test_new"] = df_test[0].rolling(window=5, min_periods=1).max()   
            df_pred["y_pred_new"] = df_pred[0].rolling(window=5, min_periods=1).max()   

            y_true = np.where(df_test["y_test_new"] >= 0.1, 1, 0)
            y_scores = y_pred


            fpr, tpr, thresholds = roc_curve(y_true, y_scores)

            optimal_idx = np.argmax(tpr - fpr)
            optimal_threshold2 = thresholds[optimal_idx]


            y_pred2 = np.where(df_pred["y_pred_new"] >= optimal_threshold2, 1, 0)
            y_test2 = np.where(df_test["y_test_new"] >= 0.1, 1, 0)

            cm = confusion_matrix(y_test2, y_pred2)

            tn = cm[0][0]
            fp = cm[0][1]
            fn = cm[1][0]
            tp = cm[1][1]
            
            precision = float(tp/(tp+fp))
            recall = float(tp/(tp+fn))
            f1 = float((2*precision*recall)/(precision+recall))
            auc_roc=str(round(roc_auc_score(y_test2, y_pred), 4))
            precision_, recall_, _ = precision_recall_curve(y_test2, y_pred)
            auc_pr=str(round(auc(recall_, precision_), 4))
            tss = float((tp/(tp+fn)) - (fp/(fp+tn)))

            df.loc[len(df)] = [col, file, '2', tp, fp, fn, tn, round(precision, 4), round(recall, 4), round(f1, 4), auc_roc, auc_pr, round(tss, 4)]


########################################################################################## 
            print("# evaluation v3")

            df_pred = pd.DataFrame(y_pred)
            df_test = pd.DataFrame(y_test)

            pred_bin = np.where(df_pred[0] >= optimal_threshold, 1, 0)
            test_bin  = np.where(df_test[0] >= 0.1, 1, 0)

            # tu je moznost to menit dala som ze v okne 6 sa ma nachadzat hodnota vacsia ako treshold minimalne 2x
            df_pred["pred_bin"] = pred_bin
            cond = df_pred["pred_bin"].rolling(window=6, min_periods=1).sum().ge(2)  


            df_test["test_bin"] = test_bin
            cond2 = df_test["test_bin"].rolling(window=6, min_periods=1).sum().ge(2) 


            df_pred["y_pred_new"] = df_pred["pred_bin"].where(cond).replace(0,1).fillna(0)
            df_test["y_test_new"] = df_test["test_bin"].where(cond2).replace(0,1).fillna(0)


            y_pred2 = df_pred["y_pred_new"] 
            y_test2 = df_test["y_test_new"] 

            cm = confusion_matrix(y_test2, y_pred2)


            tn = cm[0][0]
            fp = cm[0][1]
            fn = cm[1][0]
            tp = cm[1][1]
            
            precision = float(tp/(tp+fp))
            recall = float(tp/(tp+fn))
            f1 = float((2*precision*recall)/(precision+recall))
            auc_roc=str(round(roc_auc_score(y_test2, y_pred),4))
            precision_, recall_, _ = precision_recall_curve(y_test2, y_pred)
            auc_pr=str(round(auc(recall_, precision_),4))
            tss = float((tp/(tp+fn)) - (fp/(fp+tn)))

            df.loc[len(df)] = [col, file, '3', tp, fp, fn, tn, round(precision, 4), round(recall, 4), round(f1, 4), auc_roc, auc_pr, round(tss, 4)]


            
    df.to_csv(f"eval_multinn.csv")
    

###################
# settings
nO2+
NmE
Exospheric_temp[K]
Sun_altitude
p_Sig1
ne
SI_ind_Sig1
S4_Sig1
B
Ti
AE
1007/1007 [==============================] - 65s 64ms/step
# eval v1
# evaluation v2
# evaluation v3
AsyD
1007/1007 [==============================] - 65s 64ms/step
# eval v1
# evaluation v2
# evaluation v3
BzGSE
1007/1007 [==============================] - 68s 66ms/step
# eval v1
# evaluation v2
# evaluation v3
E
1007/1007 [==============================] - 68s 66ms/step
# eval v1
# evaluation v2
# evaluation v3
PC
1007/1007 [==============================] - 67s 66ms/step
# eval v1
# evaluation v2
# evaluation v3
Xbsn
1007/1007 [==============================] - 68s 66ms/step
# eval v1
# evaluation v2
# evaluation v3
Kp_index
f10.7_index
Dst_index
ap_index
AsyH
SymH
O2[cm^-3]
N2[cm^-3]


In [59]:
df

,col,epoch,eval_v,tp,fp,fn,tn,p,r,f1,auc-roc,auc-pr,tss
0,AE,epoch-05-mae-0.0472.hdf5,1,393,5582,149,26078,0.0658,0.7251,0.1206,0.8386,0.1248,0.5488
1,AE,epoch-05-mae-0.0472.hdf5,2,1138,8878,279,21907,0.1136,0.8031,0.1991,0.8111,0.2103,0.5147
2,AE,epoch-05-mae-0.0472.hdf5,3,616,6553,202,24831,0.0859,0.7531,0.1543,0.8129,0.1491,0.5443
3,AsyD,epoch-05-mae-0.0484.hdf5,1,393,5582,149,26078,0.0658,0.7251,0.1206,0.8386,0.1248,0.5488
4,AsyD,epoch-05-mae-0.0484.hdf5,2,1138,8878,279,21907,0.1136,0.8031,0.1991,0.8111,0.2103,0.5147
5,AsyD,epoch-05-mae-0.0484.hdf5,3,616,6553,202,24831,0.0859,0.7531,0.1543,0.8129,0.1491,0.5443
6,BzGSE,epoch-05-mae-0.0516.hdf5,1,393,5582,149,26078,0.0658,0.7251,0.1206,0.8386,0.1248,0.5488
7,BzGSE,epoch-05-mae-0.0516.hdf5,2,1138,8878,279,21907,0.1136,0.8031,0.1991,0.8111,0.2103,0.5147
8,BzGSE,epoch-05-mae-0.0516.hdf5,3,616,6553,202,24831,0.0859,0.7531,0.1543,0.8129,0.1491,0.5443
9,E,epoch-05-mae-0.0469.hdf5,1,393,5582,149,26078,0.0658,0.7251,0.1206,0.8386,0.1248,0.5488
